In [4]:
import datetime
import os
import pandas as pd
import sys
import time

sys.path.append("../src")
import localmodule

In [8]:
# Define constants.
dataset_name = localmodule.get_dataset_name()
models_dir = localmodule.get_models_dir()
oldbird_model_dir = os.path.join(models_dir, "oldbird")
units = localmodule.get_units()
clip_suppressor_modes = ["no-clip-suppressor", "clip-suppressor"]
n_thresholds = 100

In [5]:
# Print header.
start_time = int(time.time())
print(str(datetime.datetime.now()) + " Start.")
print("Evaluating Old Bird on " + dataset_name + ".")
print('pandas version: {:s}'.format(pd.__version__))
print("")

2017-08-09 05:31:24.203600 Start.
Evaluating Old Bird on BirdVox-70k.
pandas version: 0.20.3



In [27]:
# Loop over units.
unit_str = "unit01"
#for unit_str in units:    ENABLE
unit_dir = os.path.join(oldbird_model_dir, unit_str)

# Loop over clip suppressor modes.
clip_suppressor_str = "no-clip-suppressor"
#for clip_suppressor_str in clip_suppressor_modes:   ENABLE
prediction_name = "_".join(["predictions", clip_suppressor_str])
prediction_dir = os.path.join(unit_dir, prediction_name)

# Loop over thresholds.
threshold_id = 0
#for threshold_id in range(n_thresholds):    ENABLE
threshold_str = "th-" + str(threshold_id).zfill(2)

# Load Thrush prediction.
thrush_components_list = [
    dataset_name,
    "oldbird",
    "thrush",
    unit_str,
    threshold_str,
    "predictions"
]
if clip_suppressor_str == "clip_suppressor":
    thrush_components_list.append(clip_suppressor_str)
thrush_prediction_name = "_".join(thrush_components_list)
thrush_prediction_path = os.path.join(
    prediction_dir, thrush_prediction_name + ".csv")
thrush_df = pd.read_csv(thrush_prediction_path)

# Load Tseep prediction.
tseep_components_list = thrush_components_list
tseep_components_list[2] = "tseep"
tseep_prediction_name = "_".join(tseep_components_list)
tseep_prediction_path = os.path.join(
    prediction_dir, tseep_prediction_name + ".csv")
tseep_df = pd.read_csv(tseep_prediction_path)

# Merge Thrush and Tseep predictions.
merged_df = pd.concat((thrush_df, tseep_df))
merged_df = merged_df.sort_values(["Time (s)"])

# Export merged predictions as CSV.
merged_components_list = thrush_components_list
merged_components_list[2] = "merged"
merged_prediction_name = "_".join(merged_components_list)
merged_prediction_path = os.path.join(
    prediction_dir, merged_prediction_name + ".csv")
merged_df.to_csv(merged_prediction_path)